<a href="https://colab.research.google.com/github/Vixlump/5minute-lang/blob/main/A2Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#Install required package ipywidgets so I can perform some widgets in the notebook
!pip install ipywidgets
import math
import random
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown


In [36]:
# l-system creation
# expands an axiom string using rewrite rules for N iterations

def generate_lsystem(axiom, rules, iterations):
    current = axiom

    for _ in range(iterations):
        next_string = ""
        for char in current:
            next_string += rules.get(char, char)
        current = next_string

    return current


In [37]:
# turtle rendering
# Converts L-system string into drawable line segments
# Includes color and thickness variation
def draw_lsystem(instructions,
                 angle=25,
                 length=5,
                 length_scale=0.9,
                 thickness=2,
                 thickness_scale=0.8,
                 color_mode="green",
                 jitter=0.0):

    stack = []
    x, y = 0, 0
    heading = 90  # start upward
    current_length = length
    current_thickness = thickness

    lines = []

    for char in instructions:

        if char == "F":
            # Add slight randomness if desired
            jitter_angle = random.uniform(-jitter, jitter)

            rad = math.radians(heading + jitter_angle)
            new_x = x + current_length * math.cos(rad)
            new_y = y + current_length * math.sin(rad)

            lines.append((x, y, new_x, new_y,
                          current_thickness,
                          get_color(color_mode, current_length)))

            x, y = new_x, new_y

        elif char == "+":
            heading += angle

        elif char == "-":
            heading -= angle

        elif char == "[":
            stack.append((x, y, heading,
                          current_length * length_scale,
                          current_thickness * thickness_scale))

        elif char == "]":
            x, y, heading, current_length, current_thickness = stack.pop()

    return lines


In [38]:
# color style
# Provides different color palettes
def get_color(mode, depth_value):

    if mode == "green":
        return (0, 0.5 + random.random() * 0.5, 0)

    elif mode == "autumn":
        return (random.uniform(0.6, 1.0),
                random.uniform(0.2, 0.5),
                0)

    elif mode == "blue":
        return (0,
                random.uniform(0.3, 0.7),
                random.uniform(0.6, 1.0))

    else:
        return (0, 0, 0)


In [39]:

# RULE SET 1 FRACTAL TREE
tree_axiom = "F"
tree_rules = {
    "F": "FF-[-F+F+F]+[+F-F-F]"
}

# RULE SET 2 CLASSIC PLANT

plant_axiom = "X"
plant_rules = {
    "X": "F+[[X]-X]-F[-FX]+X",
    "F": "FF"
}

# RULE SET 3 DRAGON CURVE

dragon_axiom = "FX"
dragon_rules = {
    "X": "X+YF+",
    "Y": "-FX-Y"
}


In [40]:
# multi layer rendering
# Allows stacking multiple L-systems together

def render_pattern(rule_type="Tree",
                   iterations=4,
                   angle=25,
                   length=5,
                   layers=1,
                   color_mode="green",
                   jitter=0.0):

    plt.figure(figsize=(8,8))
    plt.axis("off")

    for layer in range(layers):

        if rule_type == "Tree":
            axiom = tree_axiom
            rules = tree_rules

        elif rule_type == "Plant":
            axiom = plant_axiom
            rules = plant_rules

        else:
            axiom = dragon_axiom
            rules = dragon_rules

        instructions = generate_lsystem(axiom, rules, iterations)

        lines = draw_lsystem(
            instructions,
            angle=angle + layer * 3,
            length=length,
            thickness=2,
            color_mode=color_mode,
            jitter=jitter
        )

        for x1, y1, x2, y2, t, c in lines:
            plt.plot([x1, x2], [y1, y2],
                     linewidth=t,
                     color=c)

    plt.show()


In [41]:
# user interface
# Allows parameter customization
interact(
    render_pattern,
    rule_type=Dropdown(options=["Tree", "Plant", "Dragon"]),
    iterations=IntSlider(min=1, max=6, step=1, value=4),
    angle=FloatSlider(min=10, max=45, step=1, value=25),
    length=FloatSlider(min=2, max=10, step=0.5, value=5),
    layers=IntSlider(min=1, max=3, step=1, value=1),
    color_mode=Dropdown(options=["green", "autumn", "blue"]),
    jitter=FloatSlider(min=0.0, max=10.0, step=0.5, value=0.0)
);


interactive(children=(Dropdown(description='rule_type', options=('Tree', 'Plant', 'Dragon'), value='Tree'), In…